In [47]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [48]:
# CSV-01-12
path_DrDoS_DNS = "../CICDDoS-2019/CSV-01-12/01-12/DrDoS_DNS.csv"
path_DrDoS_MSSQL = "../CICDDoS-2019/CSV-01-12/01-12/DrDoS_MSSQL.csv"
path_DrDoS_LDAP = "../CICDDoS-2019/CSV-01-12/01-12/DrDoS_LDAP.csv"
path_DrDoS_NTP = "../CICDDoS-2019/CSV-01-12/01-12/DrDoS_NTP.csv"
path_DrDoS_NetBIOS = "../CICDDoS-2019/CSV-01-12/01-12/DrDoS_NetBIOS.csv"
path_DrDoS_SNMP = "../CICDDoS-2019/CSV-01-12/01-12/DrDoS_SNMP.csv"
path_DrDoS_SSDP = "../CICDDoS-2019/CSV-01-12/01-12/DrDoS_SSDP.csv"
path_DrDoS_UDP = "../CICDDoS-2019/CSV-01-12/01-12/DrDoS_UDP.csv"
path_Syn = "../CICDDoS-2019/CSV-01-12/01-12/Syn.csv"
path_TFTP = "../CICDDoS-2019/CSV-01-12/01-12/TFTP.csv"
path_UDPLag = "../CICDDoS-2019/CSV-01-12/01-12/UDPLag.csv"

# CSV-03-11
path_LDAP = "../CICDDoS-2019/CSV-03-11/03-11/LDAP.csv"
path_MSSQL = "../CICDDoS-2019/CSV-03-11/03-11/MSSQL.csv"
path_NetBIOS = "../CICDDoS-2019/CSV-03-11/03-11/NetBIOS.csv"
path_Portmap = "../CICDDoS-2019/CSV-03-11/03-11/Portmap.csv"
path_Syn = "../CICDDoS-2019/CSV-03-11/03-11/Syn.csv"
path_UDP = "../CICDDoS-2019/CSV-03-11/03-11/UDP.csv"
path_UDPLag = "../CICDDoS-2019/CSV-03-11/03-11/UDPLag.csv"

paths_except_TFTP = [path_DrDoS_DNS, path_DrDoS_MSSQL, path_DrDoS_LDAP, path_DrDoS_NTP, path_DrDoS_NetBIOS, path_DrDoS_SNMP,
                          path_DrDoS_SSDP, path_DrDoS_UDP, path_Syn, path_UDPLag, path_LDAP, path_MSSQL, path_NetBIOS, path_Portmap, path_Syn, path_UDP, path_UDPLag]


In [57]:
featureSelectedBasedOn0Results = []

def giveInfoAboutFile(path, threshHoldPercentage, showPercentage):
        
    df = pd.read_csv(path, low_memory=False)
    
    totalCols = df.shape[1]
    totalRows = len(df)
    unnecessaryFeatureCount = 0
    unnecessaryFeatureNames = []

    for column in df:
        zerosInCol = (df[column] == 0).sum()
        if zerosInCol != 0:
            percentageOfZerosInRow = ((zerosInCol*100)/totalRows)
            
            if showPercentage:
                print(column , " - ", zerosInCol, " - ", percentageOfZerosInRow)

            if percentageOfZerosInRow > threshHoldPercentage:
                unnecessaryFeatureNames.append(column)
                unnecessaryFeatureCount = unnecessaryFeatureCount + 1
                
    print()
        
    unitResult = [path, threshHoldPercentage, unnecessaryFeatureNames]
    
    featureSelectedBasedOn0Results.append(unitResult)
    
    print()
    print("In", path, "Total features having more than ", threshHoldPercentage,
          "% zero are - ", unnecessaryFeatureCount, "out of ", totalCols)

In [ ]:
for path in paths_except_TFTP:
    giveInfoAboutFile(path, 90, False)
    
featureSelectedBasedOn0Results

In [ ]:
for path in paths_except_TFTP:
    giveInfoAboutFile(path, 90, True)
    
featureSelectedBasedOn0Results

In [ ]:
for path in paths_except_TFTP:
    giveInfoAboutFile(path_TFTP, 90, True)

featureSelectedBasedOn0Results

In [61]:
mylist = []

for chunk in pd.read_csv(path_TFTP, sep=';', chunksize=20000, low_memory=False):
    mylist.append(chunk)

big_data = pd.concat(mylist, axis=0)
del mylist

In [63]:
big_data


,"Unnamed: 0,Flow ID, Source IP, Source Port, Destination IP, Destination Port, Protocol, Timestamp, Flow Duration, Total Fwd Packets, Total Backward Packets,Total Length of Fwd Packets, Total Length of Bwd Packets, Fwd Packet Length Max, Fwd Packet Length Min, Fwd Packet Length Mean, Fwd Packet Length Std,Bwd Packet Length Max, Bwd Packet Length Min, Bwd Packet Length Mean, Bwd Packet Length Std,Flow Bytes/s, Flow Packets/s, Flow IAT Mean, Flow IAT Std, Flow IAT Max, Flow IAT Min,Fwd IAT Total, Fwd IAT Mean, Fwd IAT Std, Fwd IAT Max, Fwd IAT Min,Bwd IAT Total, Bwd IAT Mean, Bwd IAT Std, Bwd IAT Max, Bwd IAT Min,Fwd PSH Flags, Bwd PSH Flags, Fwd URG Flags, Bwd URG Flags, Fwd Header Length, Bwd Header Length,Fwd Packets/s, Bwd Packets/s, Min Packet Length, Max Packet Length, Packet Length Mean, Packet Length Std, Packet Length Variance,FIN Flag Count, SYN Flag Count, RST Flag Count, PSH Flag Count, ACK Flag Count, URG Flag Count, CWE Flag Count, ECE Flag Count, Down/Up Ratio, Average Packet Size, Avg Fwd Segment Size, Avg Bwd Segment Size, Fwd Header Length.1,Fwd Avg Bytes/Bulk, Fwd Avg Packets/Bulk, Fwd Avg Bulk Rate, Bwd Avg Bytes/Bulk, Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets, Subflow Fwd Bytes, Subflow Bwd Packets, Subflow Bwd Bytes,Init_Win_bytes_forward, Init_Win_bytes_backward, act_data_pkt_fwd, min_seg_size_forward,Active Mean, Active Std, Active Max, Active Min,Idle Mean, Idle Std, Idle Max, Idle Min,SimillarHTTP, Inbound, Label"
0,"46970,172.16.0.5-192.168.50.1-64670-64670-6,17..."
1,"164775,172.16.0.5-192.168.50.1-16405-16405-6,1..."
2,"177893,172.16.0.5-192.168.50.1-45838-45838-6,1..."
3,"82756,172.16.0.5-192.168.50.1-27049-27049-6,17..."
4,"153782,172.16.0.5-192.168.50.1-43305-27049-6,1..."
...,...
20107822,"1404,172.16.0.5-192.168.50.1-46880-80-6,172.16..."
20107823,"722,172.16.0.5-192.168.50.1-59405-80-6,192.168..."
20107824,"1249,172.16.0.5-192.168.50.1-59405-80-6,172.16..."
20107825,"724,172.16.0.5-192.168.50.1-53389-80-6,172.16...."


In [65]:
mylist = []

for chunk in pd.read_csv(path_TFTP, chunksize=20000, low_memory=False):
    mylist.append(chunk)

big_data = pd.concat(mylist, axis=0)
del mylist
big_data


,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,46970,172.16.0.5-192.168.50.1-64670-64670-6,172.16.0.5,64670,192.168.50.1,64670,6,2018-12-01 13:34:27.403713,13120080,4,...,0.0,1.0,1.0,13120078.0,0.0,13120078.0,13120078.0,0,1,TFTP
1,164775,172.16.0.5-192.168.50.1-16405-16405-6,172.16.0.5,16405,192.168.50.1,16405,6,2018-12-01 13:34:27.404124,3990342,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,TFTP
2,177893,172.16.0.5-192.168.50.1-45838-45838-6,172.16.0.5,45838,192.168.50.1,45838,6,2018-12-01 13:34:27.404125,11842236,4,...,0.0,1.0,1.0,11842234.0,0.0,11842234.0,11842234.0,0,1,TFTP
3,82756,172.16.0.5-192.168.50.1-27049-27049-6,172.16.0.5,27049,192.168.50.1,27049,6,2018-12-01 13:34:27.404177,3579644,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,TFTP
4,153782,172.16.0.5-192.168.50.1-43305-27049-6,172.16.0.5,43305,192.168.50.1,27049,6,2018-12-01 13:34:27.404228,90,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,TFTP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20107822,1404,172.16.0.5-192.168.50.1-46880-80-6,172.16.0.5,46880,192.168.50.1,80,6,2018-12-01 17:16:34.589373,1133,8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,205.174.165.72/c.php?ip=192.168.0.4,1,TFTP
20107823,722,172.16.0.5-192.168.50.1-59405-80-6,192.168.50.1,80,172.16.0.5,59405,6,2018-12-01 17:16:34.888238,229,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,TFTP
20107824,1249,172.16.0.5-192.168.50.1-59405-80-6,172.16.0.5,59405,192.168.50.1,80,6,2018-12-01 17:16:34.888470,94,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,TFTP
20107825,724,172.16.0.5-192.168.50.1-53389-80-6,172.16.0.5,53389,192.168.50.1,80,6,2018-12-01 17:16:38.374479,177,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,TFTP


In [66]:
big_data

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,46970,172.16.0.5-192.168.50.1-64670-64670-6,172.16.0.5,64670,192.168.50.1,64670,6,2018-12-01 13:34:27.403713,13120080,4,...,0.0,1.0,1.0,13120078.0,0.0,13120078.0,13120078.0,0,1,TFTP
1,164775,172.16.0.5-192.168.50.1-16405-16405-6,172.16.0.5,16405,192.168.50.1,16405,6,2018-12-01 13:34:27.404124,3990342,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,TFTP
2,177893,172.16.0.5-192.168.50.1-45838-45838-6,172.16.0.5,45838,192.168.50.1,45838,6,2018-12-01 13:34:27.404125,11842236,4,...,0.0,1.0,1.0,11842234.0,0.0,11842234.0,11842234.0,0,1,TFTP
3,82756,172.16.0.5-192.168.50.1-27049-27049-6,172.16.0.5,27049,192.168.50.1,27049,6,2018-12-01 13:34:27.404177,3579644,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,TFTP
4,153782,172.16.0.5-192.168.50.1-43305-27049-6,172.16.0.5,43305,192.168.50.1,27049,6,2018-12-01 13:34:27.404228,90,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,TFTP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20107822,1404,172.16.0.5-192.168.50.1-46880-80-6,172.16.0.5,46880,192.168.50.1,80,6,2018-12-01 17:16:34.589373,1133,8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,205.174.165.72/c.php?ip=192.168.0.4,1,TFTP
20107823,722,172.16.0.5-192.168.50.1-59405-80-6,192.168.50.1,80,172.16.0.5,59405,6,2018-12-01 17:16:34.888238,229,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,TFTP
20107824,1249,172.16.0.5-192.168.50.1-59405-80-6,172.16.0.5,59405,192.168.50.1,80,6,2018-12-01 17:16:34.888470,94,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,TFTP
20107825,724,172.16.0.5-192.168.50.1-53389-80-6,172.16.0.5,53389,192.168.50.1,80,6,2018-12-01 17:16:38.374479,177,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,TFTP


In [71]:
threshHoldPercentage = 90
showPercentage = True
path = path_TFTP

df = big_data

totalCols = df.shape[1]
totalRows = len(df)
unnecessaryFeatureCount = 0
unnecessaryFeatureNames = []

for column in df:
    zerosInCol = (df[column] == 0).sum()
    if zerosInCol != 0:
        percentageOfZerosInRow = ((zerosInCol*100)/totalRows)

        if showPercentage:
            print(column, " - ", zerosInCol, " - ", percentageOfZerosInRow)

        if percentageOfZerosInRow > threshHoldPercentage:
            unnecessaryFeatureNames.append(column)
            unnecessaryFeatureCount = unnecessaryFeatureCount + 1

print()

unitRes = [path, threshHoldPercentage, unnecessaryFeatureNames]

featureSelectedBasedOn0Results.append(unitRes)

print()
print("In", path, "Total features having more than ", threshHoldPercentage,
        "% zero are - ", unnecessaryFeatureCount, "out of ", totalCols)


featureSelectedBasedOn0Results


Unnamed: 0  -  199  -  0.0009896643729827197
 Source Port  -  865  -  0.004301807450402274
 Destination Port  -  865  -  0.004301807450402274
 Protocol  -  865  -  0.004301807450402274
 Flow Duration  -  566761  -  2.818608892945021
 Total Backward Packets  -  20065239  -  99.78820187780609
Total Length of Fwd Packets  -  163273  -  0.8119872923115958
 Total Length of Bwd Packets  -  20092232  -  99.92244313619766
 Fwd Packet Length Max  -  163273  -  0.8119872923115958
 Fwd Packet Length Min  -  171675  -  0.8537720162402431
 Fwd Packet Length Mean  -  163273  -  0.8119872923115958
 Fwd Packet Length Std  -  20097044  -  99.94637411591019
Bwd Packet Length Max  -  20092232  -  99.92244313619766
 Bwd Packet Length Min  -  20097770  -  99.94998465025584
 Bwd Packet Length Mean  -  20092232  -  99.92244313619766
 Bwd Packet Length Std  -  20102251  -  99.97226950480527
Flow Bytes/s  -  152489  -  0.7583564350339795
 Flow IAT Mean  -  566761  -  2.818608892945021
 Flow IAT Std  -  9458398

[['../CICDDoS-2019/CSV-01-12/01-12/DrDoS_DNS.csv',
  90,
  [' Total Backward Packets',
   ' Total Length of Bwd Packets',
   ' Fwd Packet Length Std',
   'Bwd Packet Length Max',
   ' Bwd Packet Length Min',
   ' Bwd Packet Length Mean',
   ' Bwd Packet Length Std',
   ' Flow IAT Std',
   ' Fwd IAT Std',
   'Bwd IAT Total',
   ' Bwd IAT Mean',
   ' Bwd IAT Std',
   ' Bwd IAT Max',
   ' Bwd IAT Min',
   'Fwd PSH Flags',
   ' Bwd PSH Flags',
   ' Fwd URG Flags',
   ' Bwd URG Flags',
   ' Bwd Header Length',
   ' Bwd Packets/s',
   ' Packet Length Std',
   ' Packet Length Variance',
   'FIN Flag Count',
   ' SYN Flag Count',
   ' RST Flag Count',
   ' PSH Flag Count',
   ' ACK Flag Count',
   ' URG Flag Count',
   ' CWE Flag Count',
   ' ECE Flag Count',
   ' Down/Up Ratio',
   ' Avg Bwd Segment Size',
   'Fwd Avg Bytes/Bulk',
   ' Fwd Avg Packets/Bulk',
   ' Fwd Avg Bulk Rate',
   ' Bwd Avg Bytes/Bulk',
   ' Bwd Avg Packets/Bulk',
   'Bwd Avg Bulk Rate',
   ' Subflow Bwd Packets',
   ' S

In [72]:
for case in featureSelectedBasedOn0Results:
    print(case[0], "---", len(case[2]))
    print()

../CICDDoS-2019/CSV-01-12/01-12/DrDoS_DNS.csv --- 48

../CICDDoS-2019/CSV-01-12/01-12/DrDoS_MSSQL.csv --- 48

../CICDDoS-2019/CSV-01-12/01-12/DrDoS_LDAP.csv --- 48

../CICDDoS-2019/CSV-01-12/01-12/DrDoS_NTP.csv --- 43

../CICDDoS-2019/CSV-01-12/01-12/DrDoS_NetBIOS.csv --- 48

../CICDDoS-2019/CSV-01-12/01-12/DrDoS_SNMP.csv --- 48

../CICDDoS-2019/CSV-01-12/01-12/DrDoS_SSDP.csv --- 43

../CICDDoS-2019/CSV-01-12/01-12/DrDoS_UDP.csv --- 43

../CICDDoS-2019/CSV-03-11/03-11/Syn.csv --- 23

../CICDDoS-2019/CSV-03-11/03-11/UDPLag.csv --- 30

../CICDDoS-2019/CSV-03-11/03-11/LDAP.csv --- 48

../CICDDoS-2019/CSV-03-11/03-11/MSSQL.csv --- 48

../CICDDoS-2019/CSV-03-11/03-11/NetBIOS.csv --- 48

../CICDDoS-2019/CSV-03-11/03-11/Portmap.csv --- 48

../CICDDoS-2019/CSV-03-11/03-11/Syn.csv --- 23

../CICDDoS-2019/CSV-03-11/03-11/UDP.csv --- 43

../CICDDoS-2019/CSV-03-11/03-11/UDPLag.csv --- 30

../CICDDoS-2019/CSV-01-12/01-12/TFTP.csv --- 46

